In [0]:
from pyspark.sql import functions as f

In [0]:
%run ./../../ingestion/structs/nb_schemas

In [0]:
# Parâmetros de entrada para a tabela Silver
src_catalog_name    = 'silver'
src_schema_name     = 'refined'
src_table_name      = 'global_superstore'

# Parâmetros da camada Gold
gold_catalog_name   = 'gold'
gold_schema_name    = 'analytics'
gold_table_prefix   = 'global_superstore'

gold_parquet_tables = '/Volumes/gold/analytics/parquets_files'

In [0]:

table_path = f"{src_catalog_name}.{src_schema_name}.{src_table_name}"
spark.sql(f"CREATE CATALOG IF NOT EXISTS {gold_catalog_name}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {gold_catalog_name}.{gold_schema_name}")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {gold_catalog_name}.{gold_schema_name}.parquets_files")
df = spark.read.table(table_path)

In [0]:
df = spark.read.table(table_path)

#### Evolução anual das vendas, quantidades vendidas e preço médio por transação. 

In [0]:
### Adicione uma coluna calculada, por exemplo, o valor total de cada transação.
total_order_value = (
    df
    .groupBy('order_id')
    .agg(
        f.sum(f.col('sales')).cast('decimal(12,2)').alias('total_order_value')
    )
)


(
    total_order_value
    .coalesce(1)
    .write
    .format('parquet')
    .mode("overwrite")
    .save(f"{gold_parquet_tables}/total_order_value")
)
